In [6]:
class comp_node :
    def __init__(self ,val ,children=[],op="assign"):
        self.val= val 
        self.children= children  
        self.grad=0
        self.op= op
        self.backward = lambda : None     
    def __sub__(self ,other):
        if not isinstance(other,comp_node):
            other = comp_node (val = other)
        out = comp_node(val = self.val - other.val, 
                        children= [self,other],op="sub")
        def backward():
            self.grad += out.grad * (-1)
            other.grad += out.grad * (-1)
        out.backward= backward 
        return out 
    def __rsub__(self ,other):
        if not isinstance(other,comp_node):
            other = comp_node (val = other)
        # out = comp_node(val =  other.val - self.val , 
        #                 children= [self,other],op="sub")
        return other - self  
    def __add__(self ,other):
        if not isinstance(other,comp_node):
            other = comp_node (val = other)
        out = comp_node(val = self.val + other.val, 
                        children= [self,other],op="add")
        def backward(): 
            self.grad += out.grad * (1)
            other.grad += out.grad * (1)
        out.backward=backward    
        return out 
    def __radd__(self ,other):
        if not isinstance(other,comp_node):
            other = comp_node (val = other)
        out = comp_node(val = self.val + other.val, 
                        children= [self,other],op="add")
        return out
    def __mul__(self ,other):
        if not isinstance(other,comp_node):
            other = comp_node (val = other)
        out= comp_node(val = self.val * other.val ,
                       children =[self],op = "multi")
        def backword(): 
            self.grad += out.grad * other.val
            other.grad += out.grad * self.val
        out.backward = backword  
        return out  
    def __rmul__(self ,other):
        if not isinstance(other,comp_node):
            other = comp_node (val = other)
        out= comp_node(val = self.val * other.val ,
                       children =[self],op = "multi")
        return out 
    def __pow__ (self,exp):
        if not isinstance(exp,(int,float)):
             raise ValueError ("unsupported type")
        out = comp_node(val = self.val **exp ,
                        children = [self],op="power")
        def backward():
            self.grad += out.grad * (exp * self.val**(exp-1))
        out.backward = backward
        return out  
    def __eq__ (self ,other):
        return self.val == other.val
    def __repr__(self):
        return f"op:{self.op} | val: {self.val} | children: {len(self.children)}| grad:{self.grad}" 
    def __truediv__(self, other):
        other = self.__to_comp_node(other)
        if other.val == 0:
          raise ValueError("can not divide by zero")
        out = comp_node(val=self.val / other.val, children=[self, other], op="div")
        def __backward_prop():
            # d(u/v)/dx = (v*du/dx - u*dv/dx) / v^2 ---> For the first operand 'self'
            self.grad += out.grad / other.val
            # d(u/v)/dv = -u/(v^2) ----> For the second operand 'other'
            other.grad -= out.grad * self.val / (other.val**2)
            out.backward_prop = __backward_prop
            return out
        
    def sin(self):
            out = comp_node(val=np.sin(self.val), children=[self], op="sin")
            def __backward_prop():
                self.grad += out.grad * np.cos(self.val)
            out.backward_prop = __backward_prop
            return out
        
    def cos(self):
            out = comp_node(val=np.cos(self.val), children=[self], op="cos")
            def __backward_prop():
                self.grad -= out.grad * np.sin(self.val)
            out.backward_prop = __backward_prop
            return out
            

In [7]:
from random import Random
rand = Random(5)
def generate_point (N=1000):
    return ([rand.uniform(0, 1) for xi in range(N)],
            [rand.uniform(0, 1) for yi in range(N) ])
datax ,datay = generate_point(1)
xp , yp = comp_node(val=0.3 ),comp_node(val=0.3)
def loss_graph (xp,yp, datax,datay):
    Ix ,Iy = xp- datax , yp- datay
    gx ,gy = Ix**2, Iy**2
    M = gx + gy 
    l= M ** 0.5 
    return l , [l,M,gx,gy,Ix,Iy,xp,yp]
l ,rev_topo_order = loss_graph(xp,yp,datax[0],datay[0])
rev_topo_order[0].grad = 1
print(l) ## 7agat kteer mt8zna gwa 34an kda 3mlt repr 
for i ,node in enumerate(rev_topo_order):
    node.backward()
    print(i,node) 
print(rev_topo_order[2])


op:power | val: 0.5472122517293468 | children: 1| grad:1
0 op:power | val: 0.5472122517293468 | children: 1| grad:1
1 op:add | val: 0.29944124844270203 | children: 2| grad:0.9137222319490423
2 op:power | val: 0.10426550456264216 | children: 1| grad:0.9137222319490423
3 op:power | val: 0.19517574388005984 | children: 1| grad:0.9137222319490423
4 op:sub | val: -0.32290169488970194 | children: 2| grad:-0.5900849147094943
5 op:sub | val: -0.4417869892607294 | children: 2| grad:-0.8073411877467226
6 op:assign | val: 0.3 | children: 0| grad:0.5900849147094943
7 op:assign | val: 0.3 | children: 0| grad:0.8073411877467226
op:power | val: 0.10426550456264216 | children: 1| grad:0.9137222319490423


In [11]:
assert (6 - comp_node(5)).val == -1 
# assert (comp_node(6)**2).val == 36